In [4]:
# All  import statements needed for the notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.metrics import *

In [5]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
data = pd.read_csv('/content/drive/MyDrive/balanced_data.csv')

In [8]:
data.drop(data.columns[0] , inplace=True , axis=1)

In [9]:
unknown = ['SSH-Patator','DoS slowloris','DoS Slowhttptest','Bot','Infiltration','Heartbleed']
att = data.loc[(data['Label'].isin(unknown))]

In [10]:
data.drop(att.index,axis=0 , inplace=True, errors='ignore')

In [11]:
# importing required libraries for normalizing data
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
# selecting numeric attributes columns from data
numeric_col = data.select_dtypes(include='number').columns
# using standard scaler for normalizing
std_scaler = MinMaxScaler()
def normalization(df,att,col):
  for i in col:
    arr = df[i]
    arr = np.array(arr)
    x = np.array(att[i])
    df[i] = std_scaler.fit_transform(arr.reshape(len(arr),1))
    #To use the same scaler which was used in preprocessing the train data 
    att[i] = std_scaler.transform(x.reshape(len(x),1))
  return df,att
# calling the normalization() function
data , att = normalization(data.copy(),att.copy(),numeric_col)

In [12]:
att.Label = 'unknown'
X = att.drop('Label' , axis=1)
X = X.to_numpy().reshape(-1, 83,1)

y = att.Label
X_train = data.drop('Label' , axis=1)
X_train = X_train.to_numpy().reshape(-1, 83,1)

y_train = data.Label

In [13]:
from tensorflow import keras
model = keras.models.load_model('/content/drive/MyDrive/models/model_split_softmax_cnn_model.hdf5')

In [14]:
predicted = model.predict(X)
Y_train_predicted = model.predict(X_train)

57261/57261 [==============================] - 134s 2ms/step


In [15]:
Threshold_values = [0.4 ,0.5 , 0.6 , 0.7 , 0.8 , 0.9 , 0.95]

In [16]:
my_dict = {
    0 : 'BENIGN',
    1 : 'DDoS',
    2 : 'DoS Hulk',
    3 : 'DoS GoldenEye',
    4 : 'PortScan',
    5 : 'FTP-Patator'
}
max_probability = np.amax(predicted, axis = 1)
max_probability_train = np.amax(Y_train_predicted, axis = 1)
y_prediction_index = np.argmax(Y_train_predicted, axis = 1)
y_prediction_test_label = np.vectorize(my_dict.get)(y_prediction_index)

In [40]:
RIGHT_BOOLEAN = y_train == y_prediction_test_label

In [19]:
RIGHT_BOOLEAN.shape

(1832348,)

In [20]:
data.shape

(1832348, 84)

In [21]:
max_probability_train.shape

(1832348,)

In [22]:
type(max_probability_train)

numpy.ndarray

In [ ]:
def f(x,t):
  pass

In [42]:
TOTAL = data.shape[0]
## ONLY TRAIN TEST WITHOUT UNKNOWN
for threshold in Threshold_values:
  #for each threshold value we will check out the accuracy of the model
  print('USING THRESHOLD VALUE ',threshold)
  ## Checking in train data now...
  truepredicted = np.logical_and(max_probability_train > threshold , RIGHT_BOOLEAN).sum()

  Accuracy = truepredicted/TOTAL
  print('Accuracy is ',Accuracy)

USING THRESHOLD VALUE  0.4
Accuracy is  0.7765244374976805
USING THRESHOLD VALUE  0.5
Accuracy is  0.7764982416003947
USING THRESHOLD VALUE  0.6
Accuracy is  0.7764480327972634
USING THRESHOLD VALUE  0.7
Accuracy is  0.7763519811738818
USING THRESHOLD VALUE  0.8
Accuracy is  0.7762832169435063
USING THRESHOLD VALUE  0.9
Accuracy is  0.7760660092951776
USING THRESHOLD VALUE  0.95
Accuracy is  0.7759328468173077


In [41]:
TOTAL = data.shape[0]+att.shape[0]
## WITH BOTH TRAIN TEST && UNKNOWN SET
for threshold in Threshold_values:
  #for each threshold value we will check out the accuracy of the model
  print('USING THRESHOLD VALUE ',threshold)
  ## Checking in UNKNOWN SET
  truepredicted = (max_probability < threshold).sum()
  
  ## Checking in train data now...
  truepredicted += np.logical_and(max_probability_train > threshold , RIGHT_BOOLEAN).sum()


  Accuracy = truepredicted/TOTAL
  print('Accuracy is ',Accuracy)

USING THRESHOLD VALUE  0.4
Accuracy is  0.7688988648988174
USING THRESHOLD VALUE  0.5
Accuracy is  0.7693071707912222
USING THRESHOLD VALUE  0.6
Accuracy is  0.7693071707912222
USING THRESHOLD VALUE  0.7
Accuracy is  0.7693001496581519
USING THRESHOLD VALUE  0.8
Accuracy is  0.7707708069928325
USING THRESHOLD VALUE  0.9
Accuracy is  0.7708258958830776
USING THRESHOLD VALUE  0.95
Accuracy is  0.7707659462083991


In [45]:
TOTAL = att.shape[0]
## WITH UNKNOWN SET ONLY
for threshold in Threshold_values:
  #for each threshold value we will check out the accuracy of the model
  print('USING THRESHOLD VALUE ',threshold)
  ## Checking in UNKNOWN SET
  truepredicted = (max_probability < threshold).sum()
  
  ## Checking in train data now...
  #truepredicted = np.logical_and(max_probability_train > threshold , RIGHT_BOOLEAN).sum()


  Accuracy = truepredicted/TOTAL
  print('Accuracy is ',Accuracy)

USING THRESHOLD VALUE  0.4
Accuracy is  0.04134340015620932
USING THRESHOLD VALUE  0.5
Accuracy is  0.08320749804738349
USING THRESHOLD VALUE  0.6
Accuracy is  0.08799791720906014
USING THRESHOLD VALUE  0.7
Accuracy is  0.09648529028898724
USING THRESHOLD VALUE  0.8
Accuracy is  0.24483207498047382
USING THRESHOLD VALUE  0.9
Accuracy is  0.2708669617287165
USING THRESHOLD VALUE  0.95
Accuracy is  0.27779224160374905
